# Psychic
This notebook covers how to load documents from the [Psychic python library](https://pypi.org/project/psychicapi/) and use them for question answering. See [here](../../../../ecosystem/psychic.md) for more details.

## Prerequisites
1. Follow the Quick Start section in [this document](../../../../ecosystem/psychic.md)
2. Log into the [Psychic dashboard](https://dashboard.psychic.dev/) and get your secret key
3. Install the frontend react library into your web app and have a user authenticate a connection. The connection will be created using the connection id that you specify.

In [24]:
!pip install psychicapi
!pip install langchain
!pip install openai 
!pip install chromadb
!pip install tiktoken


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
  Using cached chromadb-0.3.25-py3-none-any.whl (86 kB)
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
  Using cached hnswlib-0.7.0.tar.gz (33 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached onnxruntime-1.15.0-cp310-cp310-macosx_11_0_arm64.whl (6.1 MB)


  Using cached overrides-7.3.1-py3-none-any.whl (17 kB)
  Using cached clickhouse_connect-0.5.25-cp310-cp310-macosx_11_0_arm64.whl (227 kB)
  Using cached duckdb-0.8.0-cp310-cp310-macosx_11_0_arm64.whl (12.6 MB)
  Using cached posthog-3.0.1-py2.py3-none-any.whl (37 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.7/364.7 kB 5.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 17.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.9 MB/s 


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [1]:
import os

## Loading documents

Use the `get_documents` function to load in documents from a connection. Each connection has a connector id (corresponding to the SaaS app that was connected) and a connection id (which you passed in to the frontend library).

In [28]:
from dotenv import load_dotenv
load_dotenv()
from langchain.docstore.document import Document
from psychicapi import Psychic, ConnectorId
from bs4 import BeautifulSoup
import datetime

# Create a document loader for google drive. We can also load from other connectors by setting the connector_id to the appropriate value e.g. ConnectorId.notion.value
# This loader uses our test credentials
psychic = Psychic(secret_key=os.getenv("PSYCHIC_SECRET_KEY"))

raw_messages = psychic.get_conversations(connector_id=ConnectorId.slack, account_id="slack_test")
crammed_messages = []
channel_id = None
channel_name = None
for message in raw_messages:
    # Message content is provided as HTML in order to preserve markup like links, tables, images, etc.
    channel_id = message["channel"]["id"]
    channel_name = message["channel"]["name"]
    content = BeautifulSoup(message["content"]).get_text()
    time = datetime.datetime.fromtimestamp(float(message["timestamp"])).strftime("%m/%d/%y:%H:%M")
    crammed_messages.append("({}) ".format(time) + message["sender"]["name"] + ": " + content)
# print(crammed_messages)

docs = [Document(page_content="\n".join(crammed_messages), metadata={"source": channel_id, "source_name": channel_name})]
print(docs)


[Document(page_content='(05/31/23:12:17) jason: nvm adding it now\n(05/31/23:10:53) jason: i.e. add get_conversations\n(05/31/23:10:53) jason: did you already make the changes to the python package for slack? I want to make a post about it today', metadata={'source': 'C04DWRUG83T', 'source_name': 'engineering'})]


## Converting the docs to embeddings 

We can now convert these documents into embeddings and store them in a vector database like Chroma

In [29]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)


embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

## Question answering

Now we use OpenAI to ask questions over the

In [32]:
chain = RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="stuff", retriever=docsearch.as_retriever())
chain({"question": "what did jason want to do on 05/31/2023?"}, return_only_outputs=True)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'answer': ' Jason wanted to add the get_conversations feature to the Python package for Slack.\n',
 'sources': 'C04DWRUG83T'}